In [1]:
import pandas as pd
from tqdm import tqdm
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm
np.random.seed(42)
tf.random.set_seed(42)
from keras.callbacks import EarlyStopping 
from keras.models import load_model

In [2]:
def training_dataset_(
    file_path,
    image_per_batch,
    start_row,
    ):
    df = pd.read_csv(file_path)[start_row:start_row+image_per_batch]
    
    df = df.drop('Unnamed: 0',axis=1)
    
    y = get_data_from_local(df)
    x = df.drop('image',axis=1)
    x = x.values
    
    return x , y

In [3]:
def get_data_from_local(X_train):
    X_data =[]
    for file in tqdm(X_train['image']):
        face = cv2.imread(file,cv2.IMREAD_COLOR)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face =cv2.resize(face, (200,200))
#         face = np.expand_dims(face, axis=2)
        face = np.asarray(face)
        face = face.astype('float32')
        face /= 255
        X_data.append(face)
    return np.array(X_data)

## model

In [4]:
from keras.layers import Conv2D , MaxPool2D ,ZeroPadding2D , UpSampling2D ,Reshape, Input ,Dense, Activation,Flatten
from keras.utils import np_utils
from keras.models import Model

def model(number_of_label):
    inp = Input(shape = (number_of_label,))
    
    fc1 = Dense(500,activation="sigmoid")(inp)
    
    fc2 = Dense(2000,activation="sigmoid")(fc1)
    
    fc3 = Dense(4000,activation="sigmoid")(fc2)

    fc4 = Dense(8836,activation="sigmoid")(fc3)

    res1 = Reshape((47,47,4))(fc4)
    zrp1 = ZeroPadding2D((1,1))(res1)

    conv4 = Conv2D(16,(3,3),padding="same" , activation ="relu")(zrp1)
    ups1 = UpSampling2D((2,2))(conv4)
    
    conv8 = Conv2D(16,(3,3),padding="same" , activation ="relu")(ups1)
    ups7 = UpSampling2D((2,2))(conv8)

    zrp2 = ZeroPadding2D((1,1))(ups7)
    conv5 = Conv2D(32,(3,3),padding="same" , activation ="relu")(zrp2)

    zrp3 = ZeroPadding2D((1,1))(conv5)
    conv6 = Conv2D(3,(3,3),padding="same" , activation="relu")(zrp3)
    c_a_e = Model(inputs=inp, outputs=conv6)
    c_a_e.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    c_a_e.summary()
    return c_a_e

## training

In [5]:
def training(
    start_row,
    number_of_labels,
    file_path,
    image_per_batch,
    max_limit,
    path_of_model
):

    checkpoint = ModelCheckpoint(filepath=path_of_model,
                             monitor='val_accuracy',
                             save_format='tf',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )
    early_stopping_monitor = EarlyStopping(patience=2,verbose=1) 
    
    for x in range(start_row,max_limit,image_per_batch):
        
        print()
        print('**********{} -- {} out of {}************'.format(x,x+image_per_batch,max_limit))
        print()
    
        X_train , y_train   =   training_dataset_(
                        file_path = file_path,
                        start_row = x,
                        image_per_batch = image_per_batch,
        )
        
        if x != 0:
            final_cnn = load_model(path_of_model)
        else:
            final_cnn = model(number_of_labels)
            
        try:
            final_cnn_history = final_cnn.fit(x=X_train,
                                          y=y_train,
                                          batch_size=64,
                                          validation_split=0.5,
                                          epochs=10,
                                          callbacks=[checkpoint,early_stopping_monitor],
                                          shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                                         )
        except:
            continue


In [6]:
training(
    start_row = 0,
    number_of_labels = 3,
    file_path = 'main_dataset.csv',
    image_per_batch = 600,
    max_limit = 1000,
    path_of_model = 'final_cnn_checkpoint.tf'
)

  1%|          | 7/600 [00:00<00:08, 68.58it/s]


**********0 -- 600 out of 1000************



100%|██████████| 600/600 [00:06<00:00, 89.46it/s]


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 500)               2000      
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              1002000   
_________________________________________________________________
dense_2 (Dense)              (None, 4000)              8004000   
_________________________________________________________________
dense_3 (Dense)              (None, 8836)              35352836  
_________________________________________________________________
reshape (Reshape)            (None, 47, 47, 4)         0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 49, 49, 4)        

  2%|▏         | 9/481 [00:00<00:05, 89.26it/s]


**********600 -- 1200 out of 1000************



100%|██████████| 481/481 [00:05<00:00, 89.29it/s]


Epoch 1/10
4/4 [==============================] - ETA: 0s - loss: 0.0777 - accuracy: 0.7589
Epoch 00001: val_accuracy improved from 0.74504 to 0.75012, saving model to final_cnn_checkpoint.tf
INFO:tensorflow:Assets written to: final_cnn_checkpoint.tf/assets
4/4 [==============================] - 28s 7s/step - loss: 0.0777 - accuracy: 0.7589 - val_loss: 0.0742 - val_accuracy: 0.7501
Epoch 2/10
4/4 [==============================] - ETA: 0s - loss: 0.0741 - accuracy: 0.7505
Epoch 00002: val_accuracy did not improve from 0.75012
4/4 [==============================] - 17s 4s/step - loss: 0.0741 - accuracy: 0.7505 - val_loss: 0.0730 - val_accuracy: 0.7317
Epoch 3/10
4/4 [==============================] - ETA: 0s - loss: 0.0740 - accuracy: 0.7481
Epoch 00003: val_accuracy did not improve from 0.75012
4/4 [==============================] - 17s 4s/step - loss: 0.0740 - accuracy: 0.7481 - val_loss: 0.0717 - val_accuracy: 0.7487
Epoch 4/10
4/4 [==============================] - ETA: 0s - loss: 0